In [115]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import accuracy_score

In [116]:
data = pd.read_csv('diabetes.csv')
print(data)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

In [117]:
X = data.drop("Outcome", axis=1)
y = data["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [118]:
bst = XGBClassifier(n_estimators=100, max_depth=2, learning_rate=3, objective='binary:logistic', reg_alpha=30, reg_lambda=80, booster='dart')
start = time.time()
bst.fit(X_train, y_train)
end = time.time()

y_pred = bst.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Точность: {acc}")
print(f"Время обучения: {end - start} секунд")

Точность: 0.7359307359307359
Время обучения: 0.19712495803833008 секунд


подберем гиперпараметры используя Random Search

In [119]:
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

In [120]:
bst = XGBClassifier()
params = dict(num_estimators=range(10, 210, 10), max_depth=range(1, 21), learning_rate = range(1,10), reg_alpha=range(0, 100, 10), reg_lambda=range(0, 100, 10))
clf = RandomizedSearchCV(bst, params, random_state=0)
start = time.time()
clf.fit(X_train, y_train)
end = time.time()
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Подобранные гиперпараметры: {clf.best_params_}")
print(f"точность: {acc}")
print(f"Время обучения: {end-start} секунд ")

Подобранные гиперпараметры: {'reg_lambda': 50, 'reg_alpha': 10, 'num_estimators': 40, 'max_depth': 17, 'learning_rate': 4}
точность: 0.7792207792207793
Время обучения: 1.1696550846099854 секунд 


теперь подберем гиперпараметры, используя алгоритм TPE и библиотеку Hyperopt

In [121]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

определим выбор наших гиперпараметров

In [122]:
space = {'max_depth': hp.choice('max_depth', range(1, 21)),
         'n_estimators': hp.choice('n_estimators', range(10, 210, 10)),
         'reg_lamda':  hp.choice('reg_lambda', range(0, 100, 10)),
         'reg_alpha': hp.choice('reg_alpha', range(0, 100, 10)),
         'learning_rate': hp.choice('learning_rate', range(1,10))}

In [123]:

def objective(params):
    clf = XGBClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return {'loss': -accuracy_score(y_test, y_pred), 'status': STATUS_OK}

In [124]:
trials = Trials()
start = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials,
            show_progressbar=True)
end = time.time()
best_params = {'max_depth': list(range(1, 21))[best['max_depth']],
               'n_estimators': list(range(10, 210, 10))[best['n_estimators']],
               'reg_lambda': list(range(0, 100, 10))[best['reg_lambda']],
               'reg_alpha': list(range(0, 100, 10))[best['reg_alpha']],
               'learning_rate': list(range(1,10))[best['learning_rate']]}
print(f"Подобранные гиперпараметры: {best_params}")
print(f"точность: {-trials.best_trial['result']['loss']}")
print(f"Время обучения: {end - start} секнуд")

100%|██████████| 50/50 [00:01<00:00, 30.54trial/s, best loss: -0.7965367965367965]
Подобранные гиперпараметры: {'max_depth': 11, 'n_estimators': 110, 'reg_lambda': 80, 'reg_alpha': 20, 'learning_rate': 3}
точность: 0.7965367965367965
Время обучения: 1.6471788883209229 секнуд


подводя итог и сравнивая полученные результаты:
через алгоритм TPE мы получили самую высокую точность за самое короткое время(не считая резульат, полученный в лабораторной 5.2).
С помощью Random Search мы тоже получили хорошую точность за короткое время, но у TPE результаты все таки лучше.
Полученный по итогу набор дучших гиперпараметров довольно сильно различается.
В первом случае имеем большую глубину и маленький набор деревьев, во втором случае все наоборот, глубина меньше, количество деревьев больше. Значения же reg_lambda и reg_alpha хоть и отличаются, но в обоих случаях первый параметр стремится быть больше, а второй меньше